# 참고 사이트
* https://jvvp.tistory.com/1152 crud 위주
* http://jason-heo.github.io/elasticsearch/2016/07/16/elasticsearch-with-python.html#5-1-%EB%AA%A8%EB%93%A0-index-%EC%B6%9C%EB%A0%A5%ED%95%98%EA%B8%B0 

### 라이브러리 로드

In [3]:
from elasticsearch import Elasticsearch, helpers
import pandas as pd
import csv
from pprint import pprint # Json 출력 인덴트 주어 보기 편하게 출력.

### 엘라스틱 연동

In [4]:
es = Elasticsearch("")
es.info()

{'name': '6d37a750a0ac',
 'cluster_name': '-',
 'cluster_uuid': '',
 'version': {'number': '7.3.1',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '4749ba6',
  'build_date': '2019-08-19T20:19:25.651794Z',
  'build_snapshot': False,
  'lucene_version': '8.1.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

### CSV 파일 적재(대용량 방법)

In [3]:
with open(r"C:\Users\belab\Desktop\jun_test.csv") as f:
    reader = csv.DictReader(f)
    helpers.bulk(es, reader, index="munjin", doc_type="doc")
    f.close

### 검색 함수

In [5]:
def search_a(index, data=None):
    if data is None:
        data = {"match_all" : {}} # select * from Table.
    else:
        data = {"match" : data}
    body = {"query" : data}
    res = es.search(index=index, body=body)
    
    return res

* munjin 테이블에서 pheno_name에 '가슴'을 포함하는 docs 조회(단일열)
#### ≒ select * from munjin where pheno_name like '%가슴%';

In [11]:
a = search_a("munjin",{'query':'운동 후 흉통'})
pprint(a)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [],
          'max_score': None,
          'total': {'relation': 'eq', 'value': 0}},
 'timed_out': False,
 'took': 0}


<ipython-input-10-127c1468cbef>:7: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in 8.0.0. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.search(index=index, body=body)


In [14]:
pprint(es.search(index='munjin', body={"query" : {"match" : {"pheno_name" : { "query" : "운동 후 흉통", "operator" : "and"}}}}))

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': 'Qg27XnwBwCLgtTq-R_7E',
                    '_index': 'munjin',
                    '_score': 15.647085,
                    '_source': {'answer_id': 'A0002864',
                                'answer_name': '운동 후 흉통, 호흡곤란과 더불어',
                                'comment_': '',
                                'insert_dt': '44463',
                                'main': 'sub',
                                'munjin_id': '3464',
                                'naire_id': '46',
                                'naire_name': '실신 질문지',
                                'negative': '0',
                                'period_': 'present',
                                'pheno_id': '893',
                                'pheno_name': '운동 후 흉통',
                                'property': '',
                                'qualifier_id': '',
                                'qualifier_name': '',

<ipython-input-14-199e505c5c85>:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in 8.0.0. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  pprint(es.search(index='munjin', body={"query" : {"match" : {"pheno_name" : { "query" : "운동 후 흉통", "operator" : "and"}}}}))


### 적재된 모든 인덱스(테이블)명 조회

In [7]:
print(es.indices.get_alias().keys())

dict_keys(['.monitoring-kibana-7-2021.10.08', 'test_index_2', 'kibana_sample_data_ecommerce', '.monitoring-es-7-2021.10.12', '.monitoring-es-7-2021.10.07', 'munjin', '.monitoring-es-7-2021.10.11', '.monitoring-logstash-7-2021.10.08', '.monitoring-logstash-7-2021.10.12', '.monitoring-kibana-7-2021.10.12', 'clinical32', '.monitoring-es-7-2021.10.06', '.monitoring-es-7-2021.10.09', '.monitoring-kibana-7-2021.10.09', '.monitoring-logstash-7-2021.10.07', '.monitoring-kibana-7-2021.10.11', '.kibana_task_manager', '.monitoring-logstash-7-2021.10.09', '.monitoring-es-7-2021.10.08', 'test-index', '.monitoring-es-7-2021.10.10', 'news', '.kibana_1', '.monitoring-kibana-7-2021.10.10', 'nori-test', '.monitoring-logstash-7-2021.10.11', '.monitoring-kibana-7-2021.10.07', '.monitoring-kibana-7-2021.10.06', '.monitoring-logstash-7-2021.10.10', 'kibana_sample_data_flights', 'kibana_sample_data_logs', '.monitoring-logstash-7-2021.10.06', 'test_jun'])


### 인덱스 생성

In [1]:
munjin_index_setting = {
    "settings":{
        "analysis":{
            "analyzer":{
                "nori":{
                    "type":"custom",
                    "tokenizer":"nori_tokenizer",
                    "decompound":"mixed" # 복합명사 분석 방법 {None, Discard, Mixed}
                }
            }
        }
    }
    "mappings":{
        "properties":{
            "term_code":{
                "type":"keyword"
            },
            "concept_code":{
                "type":"keyword"
            },
            "name_eng":{
                "type":"text",
            },
            "name_kor":{
                "type":"text",
                "analyzer":"nori"
            }
        }
    }
}

### 인덱스 매핑 조회

In [20]:
es.indices.get_mapping("munjin")

<ipython-input-20-85984da17ad6>:1: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.get_mapping("munjin")


{'munjin': {'mappings': {'properties': {'answer_id': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'answer_name': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'comment_': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'insert_dt': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'main': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'munjin_id': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'naire_id': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'naire_name': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'negative': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'period_': {'type': 

### Analyze 테스트

In [45]:
res = es.indices.analyze(body={
    "tokenizer":"nori_tokenizer",
    "text" : "Abc ddd"
})
# tokens = [value['token'] for index, value in enumerate(res['tokens'])] # enumerate 함수는 인덱스와 값을 튜플로 반환하는 함수.
# tokens
res

{'tokens': [{'token': 'Abc',
   'start_offset': 0,
   'end_offset': 3,
   'type': 'word',
   'position': 0},
  {'token': 'ddd',
   'start_offset': 4,
   'end_offset': 7,
   'type': 'word',
   'position': 1}]}